In [1]:
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver import ChromeOptions
import time
import re
import configparser
import os
import sys

In [22]:
sys.setrecursionlimit(10000)
cf = configparser.ConfigParser()
path = os.path.abspath('./conf.properties')
cf.read(path)
username = cf.get('user', 'username')
password = cf.get('user', 'password')
price = cf.get("ticket", "price")
start_time = cf.get("ticket", "start_time")
total = cf.get("ticket", "total")
star = cf.get("ticket", "star")
address = cf.get("ticket", "address")
auto_upgrade=bool(cf.get("ticket", "auto_upgrade"))

In [3]:
option = ChromeOptions()
option.add_argument('user-agent="Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.169 Safari/537.36"')
option.add_experimental_option('excludeSwitches', ['enable-automation'])
browser=webdriver.Chrome(executable_path="/Applications/Google Chrome.app/Contents/MacOS/chromedriver", options=option)
browser.get("https://www.damai.cn/?utm_source=search&utm_medium=baidupc&utm_content=pmarket_yhcpyyb&utm_campaign=alink_bdsem_shh_shouye_8000")
browser.maximize_window()
browser.implicitly_wait(5)

In [4]:
def login():
    """登录"""
    """大麦会识别出非人工操作，从输入用户名密码开始到验证码结束人工操作"""
    #点击登录按钮,跳转到登录界面
    if browser.title=="大麦网-全球演出赛事官方购票平台-100%正品、先付先抢、在线选座！":
        browser.find_element_by_css_selector(".login-user").click()
        #进入登录的iframe，坑，竟然在一个大页面中有一个iframe
        browser.switch_to_frame("alibaba-login-box")
    browser.find_element_by_id("fm-login-id").clear()
    #输入用户名，需要替换成自己的用户名
    browser.find_element_by_id("fm-login-id").send_keys(username)
    browser.find_element_by_id("fm-login-password").clear()
    #输入密码
    browser.find_element_by_id("fm-login-password").send_keys(password)
    #判断是否需要滑动解锁验证码
    if browser.find_element_by_id("nocaptcha-password").is_displayed():
        #滑动验证码,获取平移距离,移动解锁
        start=browser.find_element_by_id("nc_1_n1z")
        inner_size=start.size["width"]
        outer_size=browser.find_element_by_class_name("nc-lang-cnt").size["width"]
        move_size=outer_size-inner_size
        ActionChains(browser).drag_and_drop_by_offset(start, move_size, 0).perform()
        time.sleep(2)#滑完了之后稍等下，让系统判断完毕

    browser.find_element_by_css_selector(".password-login").click()

In [6]:
# perl -pi -e 's/cdc_/dog_/g' /Applications/chromedriver
# 登录
login()

/Users/admin/Software/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:8: DeprecationWarning: use driver.switch_to.frame instead
  


In [7]:
def search(performer, city=None):
    """登录后首页搜索，可以选择城市"""
    if city != None:
        current_location = browser.find_element_by_class_name("city-location")
        ActionChains(browser).move_to_element(current_location).perform()
        cities = browser.find_elements_by_class_name("name-city")
        for city in cities:
            if city.text == city:
                city.click()
                break 
    browser.find_element_by_class_name("input-search").clear()
    browser.find_element_by_class_name("input-search").send_keys(performer)
    browser.find_element_by_class_name("btn-search").click()

In [8]:
# 搜索
search(star, address)

In [9]:
def buy_detail(city):
    """进入购买详情页"""
    browser.find_element_by_partial_link_text(city).click()

In [10]:
# 跳转指定城市的购买界面
buy_detail(address)

In [11]:
# 切换窗口
def switch_window():
    for window in browser.window_handles:
        if window != browser.current_window_handle:
            buy_page_window = window
            browser.switch_to_window(window)
            break

In [12]:
switch_window()

/Users/admin/Software/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:6: DeprecationWarning: use driver.switch_to.window instead
  


In [13]:
def buy_now(total):
    """total为购买票的数目"""
    # 判断限购数量
    try:
        limit = re.search("\d",browser.find_element_by_class_name("number_right_limit").text).group(0)
        if int(total) > int(limit):
            total=limit
        browser.find_element_by_class_name("cafe-c-input-number-input").clear()
        browser.find_element_by_class_name("cafe-c-input-number-input").send_keys(total)
        browser.find_element_by_class_name("buybtn").click()
    except:
        print("获取限购数量失败,直接使用输入票数【"+str(total)+"】")
        browser.find_element_by_class_name("cafe-c-input-number-input").clear()
        browser.find_element_by_class_name("cafe-c-input-number-input").send_keys(int(total))
        browser.find_element_by_class_name("buybtn").click()

In [14]:
def choose_time_and_price(price, start_time_sort_num):
    """选择价位与时间"""
    """start_time_sort_num时间顺序，页面上的排序，开始第一个数是1"""
    
    all_items = browser.find_elements_by_class_name("select_right_list_item")
    sorts=[]
    price_levels=[]
    for item in all_items:
        if item.get_attribute("class").find("sku_item") == -1:
            sorts.append(item)
        else:
            price_levels.append(item)
         
    # 选择时间
    #print("当前时间场次总共有【"+str(len(sorts))+"】档")
    if len(sorts) < int(start_time_sort_num):
        print("输入抢票日期信息错误,将被设置成第一场演唱会的票")
        start_time_sort_num = 1 
    sorts[int(start_time_sort_num)-1].click()
                
    # 选择价位
    #print("当前价位总共有【"+str(len(price_levels))+"】档")
    if len(price_levels) < int(price):
        print("输入价位信息错误，超过系统的价位档,将被设置成最低档价位的票")
        price = 1 
    price_levels[int(price)-1].click()
    
    #返回是否可以购买
    return browser.find_element_by_class_name("buybtn").text

In [15]:
count=0
def refresh_and_buy(price, total, start_time_sort_num):
    """判断是否开售，未开售就刷新浏览器"""
    """price：价格档位"""
    """total：选择抢票数目"""
    # 是否可以购票
    txt = choose_time_and_price(price, start_time_sort_num)
    while txt=="即将开抢":
        browser.refresh()
        txt = choose_time_and_price(price, start_time_sort_num)
    if txt == "提交缺货登记":
        print("该档次票已被抢完")
        if auto_upgrade:
            global count
            print("自动提升票价档次")
            price = str(int(price) + 1)
            refresh_and_buy(price, total, start_time_sort_num)
            count+=1
        #如果启用自动升档门票价格的话，抢到票后也需要退出当前程序片段
    if count == 0:
        buy_now(total)
    
    # 点击立即购买出错跳到页面，需要添加判断http://err.taobao.com/error2.html
    # 亲，同一时间下单人数过多，建议您稍后再试，会卡住
    while browser.current_url == "http://err.taobao.com/error2.html" or browser.page_source.find("next-row error-msg") > 0:
        browser.back()
        count=0
        browser.refresh()
        refresh_and_buy(price, total, start_time_sort_num)

In [19]:
def submit_order(total):
    """提交订单"""
    """判断是否有限制一个身份证只能买一张票"""
    if browser.find_element_by_class_name("ticket-buyer-title").text.find("1") > 0:
        #没有该限制
        browser.find_element_by_class_name("next-checkbox-label").click()
    else:
        #选择多位观看人
        for user in browser.find_elements_by_class_name("next-checkbox-label")[0:int(total)]:
            user.click()
      
    browser.find_elements_by_class_name("next-btn-medium")[2].click()
    

In [24]:
curr_url=browser.current_url

In [27]:
while browser.current_url == curr_url:
    #刷新及购买
    global count
    count=0
    refresh_and_buy(price=price,total=total,start_time_sort_num=start_time)
    # 页面中总共有3个“确认”，如果出现异常跳出其他按钮，“确认”会大于3个
    if browser.page_source.count("确认") > 3:
        browser.find_element_by_id("dialog-footer-2").find_element_by_tag_name("button").click()
        curr_url=browser.current_url
    else:
        # 提交订单
        
        submit_order(total)
        #如果出现 “确认” div弹窗，则退回上一层
        time.sleep(1.8)
        if browser.page_source.count("确认") > 3:
            txt = browser.find_element_by_class_name("next-feedback-title").text
            while txt.find("同一时间下单人数过多") > 0:
                browser.refresh()
                if browser.page_source.count("确认") > 3:
                    browser.find_element_by_id("dialog-footer-2").find_element_by_tag_name("button").click()
                    curr_url=browser.current_url
                else:
                    submit_order(total)
                    time.sleep(1.8)
                    if browser.page_source.count("确认") > 3:
                        txt=browser.find_element_by_class_name("next-feedback-title").text
            browser.find_element_by_id("dialog-footer-2").find_element_by_tag_name("button").click()
            curr_url=browser.current_url

该档次票已被抢完
自动提升票价档次
该档次票已被抢完
自动提升票价档次


StaleElementReferenceException: Message: stale element reference: element is not attached to the page document
  (Session info: chrome=78.0.3904.108)
